In [ ]:
# initializing otter-grader
import otter
grader = otter.Notebook()

# Homework 5: Predicting Housing Prices

## Due Date: 11:59pm Friday, May 29.**

### Collaboration Policy

Data science is a collaborative activity. While you may talk with others about the homework, we ask that you **write your solutions individually**. If you do discuss the assignments with others please **include their names** in the collaborators cell below.

**Collaborators:** *list names here*

In [1]:
import numpy as np

import pandas as pd
from pandas.api.types import CategoricalDtype

import altair as alt

# The Data

The [Ames dataset](http://jse.amstat.org/v19n3/decock.pdf) consists of 2930 records taken from the Ames, Iowa, Assessor’s Office describing houses sold in Ames from 2006 to 2010.  The data set has 23 nominal, 23 ordinal, 14 discrete, and 20 continuous variables (and 2 additional observation identifiers) --- 82 features in total.  An explanation of each variable can be found in the included `codebook.txt` file.  The information was used in computing assessed values for individual residential properties sold in Ames, Iowa from 2006 to 2010.  **Some noise has been added to the actual sale price, so prices will not match official records.**

The data are split into training and test sets with 2000 and 930 observations, respectively.

In [2]:
training_data = pd.read_csv("ames_train.csv")
test_data = pd.read_csv("ames_test.csv")

print("Training", training_data.shape)
print("Testing", test_data.shape)

As a good sanity check, we should verify that the data shape matches the description.

In [3]:
# 2000 observations and 82 features in training data
assert training_data.shape == (2000, 82)
# 930 observations and 81 features in test data
assert test_data.shape == (930, 81)
# SalePrice is hidden in the test data
assert 'SalePrice' not in test_data.columns.values
# Every other column in the test data should be in the training data
assert len(np.intersect1d(test_data.columns.values, 
                          training_data.columns.values)) == 81

The next order of business is getting a feel for the variables in our data.  The Ames data set contains information that typical homebuyers would want to know.  A more detailed description of each variable is included in [`codebook.txt`](./codebook.txt), which is included in [the same folder](./) as the notebook.  **You should take some time to familiarize yourself with the codebook before moving forward.**

In [4]:
training_data.columns.values

In [5]:
training_data.head()

Note that you cannot see the values for all columns. In order to tell Jupyter notebook to display values for all columns, we can remove the constraint on the number of columns that are being displayed.

In [6]:
pd.options.display.max_columns = None

In [7]:
training_data.head()

# Part 1: Exploratory Data Analysis

In this section, we will make a series of exploratory visualizations and interpret them.

Note that we will perform EDA on the **training data** so that information from the test data does not influence our modeling decisions. Usually, you should always work with your **training data**, leaving **testing data** until the very end, when you want to check the accuracy of the models that you trained.

### Sale Price
We begin by examining a density plot of our target variable `SalePrice`.  At the same time, we can also take a look at some descriptive statistics of this variable.

In [8]:
training_data

alt.Chart(training_data).transform_density(
    'SalePrice',
    as_=['SalePrice', 'density'],
).mark_area().encode(
    x="SalePrice:Q",
    y='density:Q',
)

In [9]:
training_data['SalePrice'].describe()

Notice how far apart lie the `min` and `max` values, especially compared to the `mean` and the median (the `50%` value). **Reminder**: the median value is the "middle" of a sorted list of numbers [1](https://www.mathsisfun.com/median.html), which means that 50\% of the values in the list lie below and above this value.

## Question 1: `SalePrice` vs. living area  <a name="q1"></a>
To check your understanding of the graph and summary statistics above, answer the following `True` or `False` questions:

1. The distribution of `SalePrice` in the training set is left-skew.
1. The mean of `SalePrice` in the training set is greater than the median.
1. At least 25% of the houses in the training set sold for more than \$200,000.00.

*The provided tests for this question do not confirm that you have answered correctly; only that you have assigned each variable to `True` or `False`.*

<!--
BEGIN QUESTION
name: q1
points: 3
-->

In [10]:
# These should be True or False
q1statement1 = ...
q1statement2 = ...
q1statement3 = ...

### SalePrice vs Gr_Liv_Area

Next, visualize the association between `SalePrice` and `Gr_Liv_Area`.  The [`codebook.txt`](./codebook.txt) file tells us that `Gr_Liv_Area` measures "above grade (ground) living area square feet."

This variable represents the square footage of the house excluding anything underground.  Some additional research (into real estate conventions) reveals that this value also excludes the garage space.

In [14]:
chart = alt.Chart(training_data).mark_point().encode(
    x='Gr_Liv_Area',
    y='SalePrice'
)

chart + chart.transform_regression('Gr_Liv_Area', 'SalePrice').mark_line(color="red")

There's certainly an association, and perhaps it's linear, but the spread is wider at larger values of both variables.  Also, there are two particularly suspicious houses above 5000 square feet that look too inexpensive for their size.  Altair automatically fits the simple linear regression based on this one predictor variable, `Gr_Liv_Area`.  Later, we'll add additional predictors to our model and manually fit the linear regression model using `sklearn` library.

## Question 2: identifying outliers <a name="q2"></a>
What are the Parcel Indentification Numbers for the two houses with `Gr_Liv_Area` greater than 5000 sqft?

*Hint*: You can answer this question in one line using [`training_data.query`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.query.html) and extracting the `PID` column.

*The provided tests for this question do not confirm that you have answered correctly; only that you have assigned `q2house1` and `q2house2` to two integers that are in the range of PID values.*

<!--
BEGIN QUESTION
name: q2
points: 2
-->

In [15]:
# Hint: You can answer this question in one line
# (q2house1, q2house2) = ...
...
print(q2house1, q2house2)

## Question 3: processing outliers <a name="q3"></a>

The codebook tells us how to manually inspect the houses using an online database called Beacon. These two houses are true outliers in this data set: they aren't the same time of entity as the rest. They were partial sales, priced far below market value. If you would like to inspect the valuations, follow the directions at the bottom of the codebook to access Beacon and look up houses by PID.

For this assignment, we will look at two ways to deal with these outliers:
remove these outliers from the data and *winzorize* the data. 

### Question 3a: removing outliers

Write a function `remove_outliers` that removes outliers from a data set based off a threshold value of a variable.  For example, `remove_outliers(training_data, 'Gr_Liv_Area', upper=5000)` should return a data frame with only observations that satisfy `Gr_Liv_Area` less than 5000.  

Inside the function, **don't change the original dataset inside the function**, since we will need the original dataset for the next question.

*Hint*: Remember that if you use `df.loc[(df["Column"] > some_value)]`, the result will return all rows for which the given boolean condition is `True`.

*The provided tests check that training_data was updated correctly, so that future analyses are not corrupted by a mistake. However, the provided tests do not check that you have implemented remove_outliers correctly so that it works with any data, variable, lower, and upper bound.*

<!--
BEGIN QUESTION
name: q3a
points: 3
-->

In [16]:
def remove_outliers(data, variable, lower=-np.inf, upper=np.inf):
    """
    Input:
      data (data frame): the table to be filtered
      variable (string): the name of the column with numerical outliers
      lower (numeric): observations with values lower than this will be removed
      upper (numeric): observations with values higher than this will be removed
    
    Output:
      a new data frame with outliers removed
      
    Note: This function should not change the contents of data.
    """  

    ## Remove outliers
    ...

    return data_no_outliers

In [19]:
training_data_cleaned = remove_outliers(training_data, 'Gr_Liv_Area', upper=5000)
training_data_cleaned.shape

Compare the summary statistics for the newly created dataframe to what we had before.

In [20]:
print("Original dataset", training_data['Gr_Liv_Area'].describe())
print("No outliers", training_data_cleaned['Gr_Liv_Area'].describe())

### Question 3b: replace outliers (winsorize)

Now, instead of removing outliers, we will replace them with another value. This process of replacing a certain number of outliers has become known as *Winsorization* / *Winsorizing the data*.

> Winsorization began as a way to "robustify" the sample mean, which is sensitive to extreme values. To obtain the Winsorized mean, you sort the data and replace the smallest $k$ values by the ($k+1$)st smallest value. You do the same for the largest values... The mean of this new set of numbers is called the Winsorized mean. If the data are from a symmetric population, the Winsorized mean is a robust unbiased estimate of the population mean.

(Source: [Winsorization: The good, the bad, and the ugly](https://blogs.sas.com/content/iml/2017/02/08/winsorization-good-bad-and-ugly.html); Retrieved May 2020)

Note that Winsorization is supposed to be *symmetric*, which means if we are replacing the $k$ largest values, we also need to replace the $k$ smallest values. For example:
If there are 10 data values, *sorted*, and $k=2$, then `D[0]` and `D[1]` are replaced by `D[2]` and the last two values (`D[8]`, `D[9]`) are replaced by `D[7]`.

*Hint*: Remember that if you use `df.loc[(df["Column"] > some_value), "Column"] = another_value`, the result will change all values in the `"Column"` of `df` for which the given boolean condition is `True` to `another_value`.

<!--
BEGIN QUESTION
name: q3b
points: 3
-->

In [21]:
def winsorize(data, variable, k=0):
    """
    Input:
      data (data frame): the table to be filtered
      variable (string): the name of the column with numerical outliers
      k (integer): the number of observations to replace
    
    Output:
      a winsorized data frame with the k outliers (smallest and largest values) replaced.
      
    Note: This function should not change the contents of data.
    """
    # Sort the values of the given variable to find the replacement values
    sorted_val = np.sort(data[variable])
    total = ...
    lower = ...
    upper = ...
    
    # Make a copy() of the data so not to change the original dataframe
    data_copy = ...

    ## Replace outliers on data_copy
    ...

    return data_copy


In [24]:
training_data_winsorized = winsorize(training_data, 'Gr_Liv_Area', k=2)
training_data_winsorized.shape

Now compare the summary statistics for the newly created dataframe to what we had before.

In [25]:
print("Original dataset:", training_data['Gr_Liv_Area'].describe())
print("No outliers:", training_data_cleaned['Gr_Liv_Area'].describe())
print("Winsorized:", training_data_winsorized['Gr_Liv_Area'].describe())

### Question 3c: when / why to use each method

Now that you have seen the effect of removing outliers versus winsorizing, briefly explain the advantage and the pitfalls of each method. Feel free to use the (above) linked resource about winsorization to explain the pros and cons in your own words.

<!--
BEGIN QUESTION
name: q3c
points: 3
manual: true
-->
<!-- EXPORT TO PDF -->

*Write your answer here, replacing this text.*

# Part 2: Feature Engineering

In this section we will create a new feature out of existing ones through a simple data transformation.

### Bathrooms

Let's create a groundbreaking new feature. Imagine that Total Bathrooms can be calculated as:

$$ \text{TotalBathrooms}=(\text{BsmtFullBath} + \text{FullBath}) + \dfrac{1}{2}(\text{BsmtHalfBath} + \text{HalfBath})$$

The actual proof is beyond the scope of this class, but we will use the result in our model.

## Question 4: adding new features <a name="q4"></a>

Write a function `add_total_bathrooms(data)` that returns a copy of `data` with an additional column called `TotalBathrooms` computed by the formula above.  **Treat missing values as zeros** (*hint*: you can use [`df.fillna()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.fillna.html)).  Remember that you can make use of vectorized code here; you shouldn't need any `for` statements. 

**Do not modify the original dataset.**

*The provided tests check that you answered correctly, so that future analyses are not corrupted by a mistake.*

<!--
BEGIN QUESTION
name: q4
points: 3
-->

In [26]:
def add_total_bathrooms(data):
    """
    Input:
      data (data frame): a data frame containing at least 4 numeric columns 
            Bsmt_Full_Bath, Full_Bath, Bsmt_Half_Bath, and Half_Bath
    """
    ...
    
training_data = add_total_bathrooms(training_data)

## Question 5a <a name="q5"></a>

Create a visualization that clearly and succintly shows that `TotalBathrooms` is associated with `SalePrice`. Avoid using `mark_point` since overplotting will be a problem. (*Hint*: Look into drawing a boxplot.)

<!--
BEGIN QUESTION
name: q5a
points: 2
manual: True
-->
<!-- EXPORT TO PDF -->

In [28]:
## Make a boxplot of sale price vs TotalBathrooms

...

## Question 5b

How does the mean sale price change with `TotalBathrooms`? How does the variance for the sale price change with `TotalBathrooms`?

<!--
BEGIN QUESTION
name: q5b
points: 1
manual: True
-->
<!-- EXPORT TO PDF -->

*Write your answer here, replacing this text.*

# Part 3: Modeling

We've reached the point where we can specify a model. But first, we will load a fresh copy of the data, just in case our code above produced any undesired side-effects. Run the cell below to store a fresh copy of the data from `ames_train.csv` in a dataframe named `full_data`. We will also store the number of rows in `full_data` in the variable `full_data_len`.

In [29]:
# Load a fresh copy of the data and get its length
full_data = pd.read_csv("ames_train.csv")
test_data = pd.read_csv("ames_test.csv")

## Question 6 <a name="q6"></a>

In the following parts, we will work with only a sample of the full data.  Complete the function `generate_training_sample` to select a random sample of the full training dataset.

To do this, first create a NumPy array named `train_indices`. `train_indices` should contain a *random* set of indices in `full_data`. By default `frac_training = 0.8` means that we will use a random 80% sample of the population.  Use `train_indices` to index into `data` to create your final `train` data frame.

<!--
BEGIN QUESTION
name: q6
points: 2
-->

In [30]:
# This makes the train-test split in this section reproducible across different runs 
# of the notebook. You do not need this line to run train_test_split in general
np.random.seed(1337)

def generate_training_sample(data, frac_training=0.8, replace=False):

    full_data_len = len(data)
    train_indices = np.random.choice(full_data_len, np.int(frac_training*full_data_len), replace=replace)
    
    # Create train by indexing into `full_data` using `train_indices`
    train = ...
    return train
    
    
train = generate_training_sample(full_data)


### Reusable Pipeline

Throughout this assignment, you should notice that your data flows through a single processing pipeline several times.  From a software engineering perspective, it's best to define functions/methods that can apply the pipeline to any dataset.  We will now encapsulate our entire pipeline into a single function `process_data_gm`.  gm is shorthand for "guided model". We select a handful of features to use from the many that are available.

In [32]:
def select_columns(data, *columns):
    """Select only columns passed as arguments and valid columns."""
    cols = [c for c in columns if c in data.columns]
    return data.loc[:, cols]

def process_data_gm(data):
    """Process the data for a guided model."""
    data = remove_outliers(data, 'Gr_Liv_Area', upper=5000)
    
    # Transform Data, Select Features
    data = add_total_bathrooms(data)
    data = select_columns(data, 
                          'SalePrice', 
                          'Gr_Liv_Area', 
                          'Garage_Area',
                          'TotalBathrooms',
                         )
    
    # Return predictors and response variables separately
    if 'SalePrice' in data.columns:
        X = data.drop(['SalePrice'], axis = 1) 
        y = data.loc[:, 'SalePrice']
    else:
        # Return none for y if SalePrice is not known
        X = data
        y = None
    
    return X, y

Now, we can use `process_data_gm` to clean our data, select features, and add our `TotalBathrooms` feature all in one step! This function also splits our data into `X`, a matrix of features, and `y`, a vector of sale prices. 

Run the cell below to feed our training and test data through the pipeline, generating `X_train`, `y_train`, and `X_test`.

In [33]:
# Pre-process our training and test data in exactly the same way
# Our functions make this very easy!
X_train, y_train = process_data_gm(full_data)

# Note that there is no y_test in the test_data
X_test, _ = process_data_gm(test_data)

### Fitting Our First Model

We are finally going to fit a model!  The model we will fit can be written as follows:

$$\text{SalePrice} = \alpha + \beta_1 \cdot \text{Gr_Liv_Area} + \beta_2 \cdot \text{Garage_Area} + \beta_3 \cdot \text{TotalBathrooms}$$

In vector notation, the same equation would be written:

$$y = \vec\beta \cdot \vec{x}$$

where $y$ is the SalePrice, $\vec\beta$ is a vector of all fitted weights (coefficients), and $\vec{x}$ contains a 1 for the bias (offset) followed by each of the feature values.

**Note:** Notice that all of our variables are continuous, except for `TotalBathrooms`, which takes on discrete ordered values (0, 0.5, 1, 1.5, ...). In this homework, we'll treat `TotalBathrooms` as a continuous quantitative variable in our model, but this might not be the best choice. We may revisit the issue in the next assignment.

## Question 7a <a name="q7a"></a>

We will use a [`sklearn.linear_model.LinearRegression`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html) object as our linear model. In the cell below, create a `LinearRegression` object and name it `linear_model`.

**Hint:** See the `fit_intercept` parameter and make sure it is set appropriately. The intercept of our model corresponds to $\alpha$ in the equation above.

*The provided tests check that you answered correctly, so that future analyses are not corrupted by a mistake.*

<!--
BEGIN QUESTION
name: q7a
points: 1
-->

In [34]:
from sklearn import linear_model as lm

linear_model = ...

## Question 7b <a name="q7b"></a>

Now, remove the commenting and fill in the ellipses `...` below with `X_train`, `y_train`, and `X_test` by generating a training sample, fitting (or "training") the linear model and predicting sale price on the test data.

With the ellipses filled in correctly, the code below should fit our linear model to the training data and generate the predicted sale prices for both the training and test datasets.

*The provided tests check that you answered correctly, so that future analyses are not corrupted by a mistake.*

<!--
BEGIN QUESTION
name: q7b
points: 2
-->

In [36]:
## Fit the linear model
...

## Generate predictions
y_fitted = ...
y_predicted = ...

## Coefficients in the linear regression model
coefs = ...
print(coefs)

## Question 7c

How much does an extra half-bathroom increase the predicted price of a house *according to our model*? 

*Hint*: a half-bathroom is a bathroom with no shower or bathtub, and in our dataset each additional half-bathroom in a house increases `TotalBathrooms` by 0.5. Put the answer *as a coefficient* in the code below.

<!--
BEGIN QUESTION
name: q7c
points: 1
-->

In [40]:
hb_increase = ...

## Question 8a <a name="q8a"></a>

Is our linear model any good at predicting house prices? Let's measure the quality of our model by calculating the **Root-Mean-Square Error (RMSE)** between our predicted house prices and the true prices stored in `SalePrice`.

$$\text{RMSE} = \sqrt{\dfrac{\sum_{\text{houses in test set}}(\text{actual price of house} - \text{predicted price of house})^2}{\text{# of houses in data set}}}$$

You can think of the RMSE as the standard deviation of the prediction errors in our model. In the cell below, write a function named `rmse` that calculates the RMSE of a model.

**Hint:** Make sure you are taking advantage of vectorized code. This question can be answered without any `for` statements.

*The provided tests check that you answered correctly, so that future analyses are not corrupted by a mistake.*

<!--
BEGIN QUESTION
name: q8a
points: 1
-->

In [42]:
def rmse(actual, predicted):
    """
    Calculates RMSE from actual and predicted values
    Input:
      actual (1D array): vector of actual values
      predicted (1D array): vector of predicted/fitted values
    Output:
      a float, the root-mean square error
    """
    ...
    
rmse(y_train, y_fitted)

## Question 8b: Understanding Sources of Uncertainty

Previously we worked with a sample of size 80%.  In this problem, assume that instead we only have access to a small random sample of size 20%.  The regression coefficients we infer when we fit the model depend on the actual _random_ sample that we get.  As a consequence, our predictions for sale price will also depend on the sample of houses we use to fit out linear regression model.  This is a source of uncertainty in our model.

The error from only observing a sample of houses is what we call *sampling variability*, since it reflects variation due to the random sample we use to train our model.  It is also sometimes referred to as "reducible error", since this source of variation can be reduced by sampling more data.  

In practice, we only get one sample, so in it can be difficult to gauge how much sampling variance we actual have.  In PSTAT 120B we learn that we can use modeling assumptions to quantify this variance.  A computational approach called [bootstrapping](https://en.wikipedia.org/wiki/Bootstrapping_\(statistics\)) is also a common method for quantifying variance and is related to the simulation we will do below.  

In this problem we will explore sampling variability in simulation by looking at how our predictions change when we use a different sample.  We will look at predictions for the first three houses in the test data.  Simulate sampling 100 different times and for each time, fit the regression model on the *training data*, and then predict on the *test* data.  Compile these predictions and make a box plot of the sampling variance of our predictions on each of these three houses.

<!--
BEGIN QUESTION
name: q8b
points: 3
-->

In [44]:
predictions = []

X_test, _ = process_data_gm(test_data)

# Explore how predictions vary depending on the sample we actually get 
# by simulating 100 random samples of size 20%
for i in range(100):
    
    ...

    y_predicted = ...
    predictions.append(y_predicted)

# Predicted house price based on each sample
prediction_df = pd.DataFrame(np.stack(predictions)).loc[:, 0:2]
prediction_df.columns=["House A", "House B", "House C"]


## Make a boxplot with Altair using the predicted house prices
...

## Question 8c: Residual Plots

One way of understanding the performance (and appropriateness) of a model is through a residual plot. Make a scatter plot of the true sale price versus the residual sale price (`y_train - y_fitted`) from our linear regression model. 

*Hint*: Create a new tidy dataframe that contains a column with the residual values corresponding to each sale price (`'SalePrice'`).

<!--
BEGIN QUESTION
name: q8c
points: 3
-->

In [46]:
...

Ideally, we would see a horizontal line of points at 0 (perfect prediction!). The next best thing would be a homogenous set of points centered at 0. However, most of this residual error in our predictions would not dissapear with more data-- this is because the linear regression model is imperfect.  Our model is probably too simple. In particular, notice that the most expensive homes are systematically more expensive than our prediction. 

Error that does not dissapear with larger training size is known as "irreducible error" and can be due to models which are too simple or a lack of the relevant predictor variables in our dataset.

In practice we need to keep in mind reducible error (e.g. the sampling variability) as well as the irreducible error due to missing important features or models which aren't exactly correct (models are almost never _exactly_ correct!)

## Question 8d <a name="q8c"></a>

Based on question 6 and 8b, do you think your predictions would improve more if you collected more data to train your linear regression model on or used a more sophisticated model?  List one way in which you could improve upon the linear regression model.


<!--
BEGIN QUESTION
name: q8d
points: 2
manual: True
-->
<!-- EXPORT TO PDF -->

*Write your answer here, replacing this text.*

Congratulations! You are finished with this homework.

# Running Built-in Tests
1. All tests are in `tests` directory
1. Each python file in `tests` is a test
1. `grader.check('testname')` runs test `'testname'`, e.g. `'q1'`
1. `grader.check_all()` runs all visible tests

In [ ]:
# Run built-in checks
grader.check_all()

In [ ]:
# Generate pdf in classic notebook (does not work in JupyterLab)
import nb2pdf
nb2pdf.convert('hw5.ipynb')

# To generate pdf using command-line, run in terminal,
# nb2pdf hw5.ipynb

# Submission Checklist
1. Check filename is 'hw5.ipynb'
1. Save file to confirm all changes are on disk
1. Run *Kernel > Restart & Run All* to execute all code from top to bottom
1. Check `grader.check_all()` output
1. Save file again to write any new output to disk
1. Check generated pdf that all responses are displayed correctly
1. Submit to Gradescope